### Meta

In [1]:
# Autoreload modules without having to restart the notebook kernel.
# hi bilbo
%load_ext autoreload
%autoreload 2


# Plotting code stolen from Georg's notebook.
import matplotlib.pyplot as plt

%matplotlib inline
font = {'family': 'DejaVu Sans',
        'weight': 'bold',
        'size': 32}
plt.rc('font', **font)

### Imports

In [2]:
import sys
sys.path.insert(1, "..\\")

import random

#import markov_clustering as mcl
import networkx as nx

# Personal libraries
import lib.cluster
import lib.graph
import lib.plot
import lib.files

In [3]:
### Constants

In [4]:
ICP55 = '4932.YER078C'
THRESHOLD = 900
MCL_INFLATION = 2  # The parameter controlling Markov Clustering

# ICP55's inviable neighbour.
YHC1 = '4932.YLR298C'
